In [1]:
%autosave 0

Autosave disabled


In [8]:
import tensorflow as tf
from tensorflow import keras

In [9]:
model = keras.models.load_model('clothing-model.h5')

In [15]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input
import numpy as np

img = load_img('pants.jpg', target_size=(299, 299))

x = np.array(img)
X = np.array([x])

X = preprocess_input(X)

In [17]:
X.shape

(1, 299, 299, 3)

In [19]:
preds = model.predict(X)

1/1 [==============================] - 0s 58ms/step


In [20]:
preds

array([[-1.8798642, -4.756312 , -2.3595333, -1.0892643,  9.903785 ,
        -2.8261814, -3.6483104,  3.2411559, -2.6120963, -4.852037 ]],
      dtype=float32)

In [21]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [22]:
dict(zip(classes, preds[0]))

{'dress': -1.8798642,
 'hat': -4.756312,
 'longsleeve': -2.3595333,
 'outwear': -1.0892643,
 'pants': 9.903785,
 'shirt': -2.8261814,
 'shoes': -3.6483104,
 'shorts': 3.2411559,
 'skirt': -2.6120963,
 't-shirt': -4.852037}

## Convert keras to tf-lite

In [25]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('clothing-model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

2024-12-12 20:11:30.019626: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,100]
	 [[{{node inputs}}]]
2024-12-12 20:11:31.569893: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,100]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: /tmp/tmpqvrv9h8d/assets


INFO:tensorflow:Assets written to: /tmp/tmpqvrv9h8d/assets
2024-12-12 20:11:39.769799: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-12-12 20:11:39.769822: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-12-12 20:11:39.769974: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpqvrv9h8d
2024-12-12 20:11:39.788914: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-12-12 20:11:39.788937: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpqvrv9h8d
2024-12-12 20:11:39.862644: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-12-12 20:11:40.141675: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /tmp/tmpqvrv9h8d
2024-12-12 20:11:40.219591: I tensorflow/cc/saved_model/loader.cc:314] SavedModel

In [26]:
!ls -lh

total 163M
-rw-rw-r-- 1 kabs kabs  83M Dec  7  2021 clothing-model.h5
-rw-rw-r-- 1 kabs kabs  81M Dec 12 20:11 clothing-model.tflite
-rw-rw-r-- 1 kabs kabs  23K Dec 12 20:01 pants.jpg
-rw-rw-r-- 1 kabs kabs 7.9K Dec 12 20:11 Untitled.ipynb


In [27]:
import tensorflow.lite as tflite

In [28]:
interpreter = tflite.Interpreter(model_path='clothing-model.tflite')
interpreter.allocate_tensors()

# check details for input
interpreter.get_input_details()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


[{'name': 'serving_default_input_8:0',
  'index': 0,
  'shape': array([  1, 299, 299,   3], dtype=int32),
  'shape_signature': array([ -1, 299, 299,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [38]:
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

# initialize input
interpreter.set_tensor(input_index, X)

# invoke the interpreter
interpreter.invoke()

# fetch output
preds = interpreter.get_tensor(output_index)
preds

array([[-1.8798664, -4.7563086, -2.3595314, -1.0892626,  9.903785 ,
        -2.826182 , -3.648309 ,  3.2411585, -2.612095 , -4.852035 ]],
      dtype=float32)

In [48]:
a = dict(zip(classes, preds[0])) # similar output as before

## Removing tf dependency

In [41]:
from PIL import Image

In [51]:
with Image.open('pants.jpg') as img:
    img = img.resize((299, 299), Image.NEAREST)

In [52]:
def preprocess_input_lite(inp):
    inp /= 127.5
    inp -= 1.
    return inp

In [53]:
x = np.array(img, dtype='float32')
X = np.array([x])

X = preprocess_input_lite(X)

In [54]:
# initialize input
interpreter.set_tensor(input_index, X)

# invoke the interpreter
interpreter.invoke()

# fetch output
preds = interpreter.get_tensor(output_index)
preds

b = dict(zip(classes, preds[0])) # should be similar output as before
assert a == b

### Another way to remove tf dependency with processing input using keras-image-helper

In [1]:
# at this point, ive install tflite_runtime and keras-image-helper

import tflite_runtime.interpreter as tflite
from keras_image_helper import create_preprocessor

classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

interpreter = tflite.Interpreter(model_path='clothing-model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

# check details for input
interpreter.get_input_details()

# preprocessor
preprocessor = create_preprocessor('xception', target_size=(299, 299))

url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

# initialize input
interpreter.set_tensor(input_index, X)

# invoke the interpreter
interpreter.invoke()

# fetch output
preds = interpreter.get_tensor(output_index)
preds

dict(zip(classes, preds[0])) # should be similar output as before

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


{'dress': -1.8798664,
 'hat': -4.7563086,
 'longsleeve': -2.3595314,
 'outwear': -1.0892626,
 'pants': 9.903785,
 'shirt': -2.826182,
 'shoes': -3.648309,
 'shorts': 3.2411585,
 'skirt': -2.612095,
 't-shirt': -4.852035}